<a href="https://colab.research.google.com/github/klmahesh/PennGrader/blob/master/hw0/grading_backend_hw0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# last update
from datetime import datetime
now = datetime.now()
print ("Last updated: {}".format(now.strftime("%Y-%m-%d %H:%M:%S")))

Last updated: 2022-01-08 00:56:23


## Change Log:
- Edited Time: 2021-09-02 17:07:32
For Question 2b testcase, changed test case input from python list to numpy array. ```xs = [math.exp(i*2.4)/3 for i in range(3)]``` to ```xs = np.array([math.exp(i*2.4)/3 for i in range(3)])```

# PennGrader Setup - HW0
- Use this notebook to set up coding homework assignment and update test cases. 

Run the following 
 cells to initialize the PennGrader Backend.

In [2]:
!pip install PyYAML

In [3]:
import json
import dill
import base64
import types
import ast
import types
import urllib.request
#import pandas as pd
import numpy as np
from datetime import datetime
import inspect
from difflib import SequenceMatcher

from urllib.error import HTTPError

# Request types
HOMEWORK_ID_REQUEST     = 'GET_HOMEWORK_ID'
UPDATE_METADATA_REQUEST = 'UPDATE_METADATA'
UPDATE_TESTS_REQUEST    = 'UPDATE_TESTS'

def is_function(val):
    return inspect.isfunction(val)

def is_module(val):
    return inspect.ismodule(val)

def is_class(val):
    return inspect.isclass(val)

def is_external(name):
    return name not in ['__builtin__','__builtins__', 'penngrader','_sh', '__main__'] and 'penngrader' not in name


class PennGraderBackend:
    
    def __init__(self, secret_key, homework_number):
        self.secret_key = secret_key
        self.homework_number = homework_number
        self.homework_id = self._get_homework_id()
        if 'Error' not in self.homework_id:
            response  = 'Success! Teacher backend initialized.\n\n'
            response += 'Homework ID: {}'.format(self.homework_id)
            print(response)
        else:
            print(self.homework_id)
            
    def update_metadata(self, deadline, total_score, max_daily_submissions):
        request = { 
            'homework_number' : self.homework_number, 
            'secret_key' : self.secret_key, 
            'request_type' : UPDATE_METADATA_REQUEST,
            'payload' : self._serialize({
                'max_daily_submissions' : max_daily_submissions,
                'total_score' : total_score,
                'deadline' : deadline
            })
        }
        print(self._send_request(request, config_api_url, config_api_key))
    
            
    def update_test_cases(self):
        request = { 
            'homework_number' : self.homework_number, 
            'secret_key' : self.secret_key, 
            'request_type' : UPDATE_TESTS_REQUEST,
            'payload' : self._serialize({
                'libraries'  : self._get_imported_libraries(),
                'test_cases' : self._get_test_cases(),
            })
        }
        print(self._send_request(request, config_api_url, config_api_key))
    
    
    def _get_homework_id(self):
        request = { 
            'homework_number' : self.homework_number,
            'secret_key' : self.secret_key,
            'request_type' : HOMEWORK_ID_REQUEST,
            'payload' : self._serialize(None)
        }
        return self._send_request(request, config_api_url, config_api_key)

        
    def _send_request(self, request, api_url, api_key):
        params = json.dumps(request).encode('utf-8')
        headers = {'content-type': 'application/json', 'x-api-key': api_key}
        try:
          request = urllib.request.Request(api_url, data=params, headers=headers)
        except err:
          return 'Request builder error: {}'.format(err.read().decode("utf-8")) 
        try:
            response = urllib.request.urlopen(request)
            return '{}'.format(response.read().decode('utf-8'))
        except HTTPError as error:
            return 'Http Error: {}'.format(error.read().decode("utf-8")) 
        
    
    def _get_imported_libraries(self):
        # Get all externally imported base packages
        packages = set() # (package, shortname)
        for shortname, val in list(globals().items()):
            if is_module(val) and is_external(shortname):
                base_package = val.__name__.split('.')[0]
                if base_package != 'google' and base_package != 'yaml':
                  packages.add(base_package)
            if (is_function(val) or is_class(val)) and is_external(val.__module__):
                base_package = val.__module__.split('.')[0]
                packages.add(base_package)
        print ('Importing packages ', packages)

        # Get all sub-imports i.e import sklearn.svm etc 
        imports = set() # (module path , shortname )
        for shortname, val in list(globals().items()):
            if is_module(val) and is_external(shortname):
                if val.__name__ in packages:
                    packages.remove(val.__name__)
                if shortname != 'drive' and shortname != 'yaml':
                  imports.add((val.__name__, shortname))

        print ('Importing libraries ', imports)
        # Get all function imports 
        functions = set() # (module path , function name)
        for shortname, val in list(globals().items()):
            if is_function(val)and is_external(val.__module__):
                functions.add((val.__module__, shortname))     
        print ('Importing functions ', functions)

        return {
            'packages' : list(packages), 
            'imports' : list(imports), 
            'functions' : list(functions)
        }

    
    def _get_test_cases(self):
        # Get all function imports 
        test_cases = {}
        for shortname, val in list(globals().items()):
            try:
                if val and is_function(val) and not is_external(val.__module__) and \
                'penngrader' not in val.__module__:
                  test_cases[shortname] = inspect.getsource(val)   
                  print ('Adding case {}', shortname)
            except:
                print ('Skipping {}', shortname)
                pass
        return test_cases

    
    def _serialize(self, obj):
        '''Dill serializes Python object into a UTF-8 string'''
        byte_serialized = dill.dumps(obj, recurse = False)
        return base64.b64encode(byte_serialized).decode("utf-8")

    
    def _deserialize(self, obj):
        byte_decoded = base64.b64decode(obj)
        return dill.loads(byte_decoded)

## Backend Initialization
- For this step, you will need a *config.yaml* file, which is under the root directory of the coding homework repo.

In [5]:
import yaml

with open(r"config.yaml") as config_file:
    student_ids = []
    config = yaml.load(config_file)

    config_api_url = config['config_api_url']
    config_api_key = config['config_api_key']

    SECRET_KEY      = config['secret_id']

    # Change this for later homeworks
    HOMEWORK_NUMBER = 0

In [6]:
backend = PennGraderBackend(secret_key = SECRET_KEY, homework_number = HOMEWORK_NUMBER)

Success! Teacher backend initialized.

Homework ID: CIS_519_Spring22_HW0


## METADATA
Homework metadata:

In [8]:
TOTAL_SCORE = 35
# Beware AWS is set to UTC, so account for this in deadlines!
DEADLINE = '2022-01-24 5:00 PM'             # 11:59 PM E.D.T
MAX_DAILY_TEST_CASE_SUBMISSIONS = 100

In [9]:
backend.update_metadata(DEADLINE, TOTAL_SCORE, MAX_DAILY_TEST_CASE_SUBMISSIONS)

Success! Metadata updated.

Total HW Points: 35
Deadline: 2022-01-24 17:00:00
Max daily submissions per test case: 100



## TEST CASES

Follow the example test case below

In [10]:
# All Libs for HW Grading
import numpy as np
import math
# import seaborn
# import torch
import pandas as pd
import random
import math

# import sklearn

# Test cases
def test_eval_exps(student_func):
    max_score = 5
    n = 10
    val = list(map(int, student_func(n)))
    if (list(val) == [1, 2, 7, 20, 54, 148, 403, 1096, 2980, 8103, 22026]):
        score = max_score
    else:
        score = 0
    # (student score, max score)
    return (score, max_score)

def test_mean_variance_tuple_from_xs_1(student_func):
    import math
    max_score = 5
    xs = np.array([math.exp(i*1.9)/3 for i in range(10)])
    if int(student_func(xs)[0]) == 1046345:
        score = max_score
    else:
        score = 0
    return (score, max_score)

def test_mean_variance_tuple_from_xs_2(student_func):
    max_score = 5
    xs = np.array([math.exp(i*2.4)/3 for i in range(3)])
    if int(student_func(xs)[1]) == 18:
        score = max_score
    else:
        score = 0
    return (score, max_score)

def test_get_mean(student_func):
    max_score = 10
    # set up iris dataset. # PennGrader does not support seaborn for now, use following line to read from raw data instead
    import pandas as pd
    iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
    # run tests
    n = 8
    species = "setosa"
#     species_top_n = [5.7, 5.5, 5.2, 5.8, 5.4, 5.4, 5.1, 5.7]
    species_mean = round(student_func(iris, species, n), 2)
    if species_mean == 5.48:
        score = max_score
    else:
        score = 0
    return (score, max_score)

def test_string_to_cat1(student_func):
    max_score = 2
    # set up iris dataset
    import pandas as pd
    iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
    # run tests
    n = 8
    df2 = student_func(iris, n)
    species = "setosa"
#     species_cat = {"setosa":0, "versicolor":1, "virginica":2}[species]
    column = "sepal_width"
    column_vals_df2 = [3.5, 3.0, 3.2, 3.1, 3.6, 3.9, 3.4, 3.4, 3.2, 3.2, 3.1, 2.3, 2.8, 2.8, 3.3, 2.4, 3.3, 2.7, 3.0, 2.9, 3.0, 3.0, 2.5, 2.9]
    if df2[column].tolist() == column_vals_df2:
        score = max_score
    else:
        score = 0
    return (score, max_score)

def test_string_to_cat2(student_func):
    max_score = 3
    # set up iris dataset
    import pandas as pd
    iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
    # run tests
    n = 8
    df2 = student_func(iris, n)
    species = "setosa"
    column = "species"
#     species_cat = {"setosa":0, "versicolor":1, "virginica":2}[species]
    column_vals_df2 = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2]
    if df2[column].tolist()== column_vals_df2:
        score = max_score
    else:
        score = 0
    return (score, max_score)

def test_string_to_cat3(student_func):
    max_score = 5
    # set up iris dataset
    import pandas as pd
    iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
    # run tests
    n = 9
    df2 = student_func(iris, n)
    if np.all(df2.columns == iris.columns):
        score = max_score
    else:
        score = 0
    return (score, max_score)

In [11]:
backend.update_test_cases()

Importing packages  {'types', 'json', 'math', 'urllib', 'dill', 'ast', 'numpy', 'inspect', 'difflib', 'base64', 'random', 'pandas', 'datetime'}
Importing libraries  {('types', 'types'), ('inspect', 'inspect'), ('random', 'random'), ('urllib', 'urllib'), ('ast', 'ast'), ('dill', 'dill'), ('pandas', 'pd'), ('math', 'math'), ('json', 'json'), ('numpy', 'np'), ('base64', 'base64')}
Importing functions  set()
Adding case {} is_function
Adding case {} is_module
Adding case {} is_class
Adding case {} is_external
Adding case {} test_eval_exps
Adding case {} test_mean_variance_tuple_from_xs_1
Adding case {} test_mean_variance_tuple_from_xs_2
Adding case {} test_get_mean
Adding case {} test_string_to_cat1
Adding case {} test_string_to_cat2
Adding case {} test_string_to_cat3
Success: Test cases updated successfully.
